# Capao Urban Rate

This notebook aims to estimate the current urban growth rate for the place: Vale do Capão, Palmeiras, BA, Brasil. By using Google Earth satellite images and analyzing RGB pixel data, we can determine the growth rate in areas featuring houses, roads, construction sites, or where forests have been cleared for humans uses. For this case study, we have chosen COPERNICUS satellite images.

In [ ]:
import ee
import geemap as geemap
import pandas as pd

from utils.utils import mileseconds_to_date
from utils.features import get_coordinates
from utils.contants import (
  PROJECT, 
  GEO_POINT,
  GEO_PLACE,
  BANDS_STANDARD,
  BANDS_LANDSET_7,
  BANDS
)

ee.Authenticate()
ee.Initialize(project=PROJECT)
geemap.ee_initialize()

# Feature Collection 

In [ ]:
# Pre process urban features
df = pd.read_csv('./data/urban_features.csv')
raw_urban = df['.geo'].tolist()
urban = [get_coordinates(i) for i in raw_urban]

urban_features_list = [
  ee.Feature(ee.Geometry.Point(urban[i][0], urban[i][1]), {'class': 1 }) for i in range(len(urban))
]

# Pre process vegetation features
df = pd.read_csv('./data/vegetation_feature.csv')
raw_vegetation = df['.geo'].tolist()
vegetation = [get_coordinates(i) for i in raw_vegetation]

vegetation_features_list = [
  ee.Feature(ee.Geometry.Point(vegetation[i][0], vegetation[i][1]), {'class': 0 }) for i in range(len(vegetation))
]

# Feature collections
urban_features = ee.FeatureCollection(urban_features_list, 'urban')
vegetation_features = ee.FeatureCollection(vegetation_features_list, 'vegetation')

feature = urban_features.merge(vegetation_features)

# feature.getInfo()

# Dataset

In [41]:
geo_point = ee.Geometry.Point(GEO_POINT)
geo_place = ee.Geometry.Polygon(GEO_PLACE)

raw_image = (
  ee.Image(
    ee.ImageCollection("LANDSAT/LC09/C02/T1")
      .filterBounds(geo_point)
      .filterDate('2023-01-01', '2023-12-01')
      .sort('CLOUDY_PIXEL_PERCENTAGE')
      .first()
  )
)

image = raw_image.select(BANDS_STANDARD).rename(BANDS)

label = 'class'

feature_collection = image.select(BANDS).sampleRegions(
  collection = feature,
  properties = [label],
  scale = 10,
  geometries = True
)

# Checking bands names
# print(image.bandNames().getInfo())

# Randon Forest Model

In [42]:
dataset = feature_collection.randomColumn()
training = dataset.filter('random <= 0.8')
validation = dataset.filter('random > 0.8')

cls = ee.Classifier.smileRandomForest(10).train(training, label, BANDS)

# store the classifier 
# %store cls

train_accuracy_standard = cls.confusionMatrix()
validation_sample_standard = validation.classify(cls)
validation_accuracy_standard = validation_sample_standard.errorMatrix(label, 'classification')

# display('Results', cls.explain().getInfo())
# display('Confusion Matrix', train_accuracy.getInfo())
# display('Training error matrix', train_accuracy)
# display('Training overall accuracy', train_accuracy.accuracy())
# display('Validation error matrix', validation_accuracy)
# display('Validation accuracy', validation_accuracy.accuracy())



# Image Series
Using Google Earch Image Collection to extract timeseries images to representate the current urban progression 

- USGS Landsat 7 Collection 2 Tier 1 Raw Scenes. Image collection started from 1997
- USGS Landsat 8 Collection 2 Tier 1 Raw Scenes. Image collection started from 2013
- USGS Landsat 9 Collection 2 Tier 1 Raw Scenes. Image collection started from 2021

Images from 1997 and 1998 aren't available in the dataset

In [60]:

image_series = {}
satelite = 'LANDSAT/LC09/C02/T1' 

for year in range(1999, 2024):
    current_bands = BANDS_LANDSET_7 if year < 2013 else BANDS_STANDARD
    if year < 2013:
        satelite = 'LANDSAT/LE07/C02/T1'
    elif 2012 < year < 2021:
        satelite = 'LANDSAT/LC08/C02/T1'

    image_collection = (ee.ImageCollection(satelite)
        .filterBounds(geo_point)
        .filterDate(f'{year}-01-01', f'{year}-12-30')
        .sort('CLOUD_COVER')
        .first()
    )

    raw_image = ee.Image(image_collection)

    # Changing the band names
    image = raw_image.select(current_bands).rename(BANDS)

    # Getting image date
    # image_raw_date = image.date().getInfo().get('value')
    # image_date = mileseconds_to_date(image_raw_date)

    # Store and Classify image
    image_series[year] = {
        'image': image, 
        # 'date': image_date,
        'landcover_classified': image.clip(geo_place).select(BANDS).classify(cls)
    }

# landcover = image_1997_1.clip(geo_place)
# current_map = geemap.Map(center=[-12.609558240448216,-41.501150593949305], zoom=15)
# current_map.addLayer(image_series[1999]['image'], {'bands': ['B1', 'B2', 'B3'], 'min': 0, 'max': 2000, "gamma": 3}, '1997')
# current_map

# Urban rate 

In [ ]:
#TODO: Create a datase with the urban area amount per year
#TODO: Create a Regression model to predict the urban area amount for next 10 years using the model. 

# Map view

In [61]:
# capao_landcover = image_series[2000]['landcover_classified'].clip(geo_place)

# landcover_area = ee.Image.pixelArea().addBands(capao_landcover)

# feature_areas = landcover_area.reduceRegions(
#   reducer = ee.Reducer.sum().group({
#     "groupField": 1,
#     "groupName": "class"
#   }),
#   geometry=feature,
#   scale=10,
#   maxPixel=1e90
# )

map = geemap.Map(center=[-12.609558240448216,-41.501150593949305], zoom=13)

vis_params = {
  "bands": ["Blues", "Green", "Red"],  
  min: 0, 
  max: 2000, 
  "gamma": 3
}

class_vis = {
    'min': 0,
    'max': 10,
    'palette': [
        'green',
        'ffbb22',
        'ffff4c',
        'f096ff',
        'fa0000',
        'b4b4b4',
        'f0f0f0',
        '0064c8',
        '0096a0',
        '00cf75',
        'fae6a0',
    ],
}

year = 2023

# map.add_layer(image, vis_params, 'Polygon')
map.add_layer(image_series[year]['image'], vis_params, 'view')
map.add_layer(image_series[year]['landcover_classified'], class_vis, 'classified', opacity=0.15)

# Display the map
map


Map(center=[-12.609558240448216, -41.501150593949305], controls=(WidgetControl(options=['position', 'transpare…